### Strategy where the Day-ahead bid commitment is just the forecast

In [ ]:
""" Loading packages """
import pandas as pd
import numpy as np
from datetime import datetime
import sklearn as skl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from matplotlib.pyplot import figure
import scipy.stats as ss

%matplotlib inline
sns.set_style('whitegrid')

import math
import random
import pyomo.opt
from pyomo.core import Var
from pyomo.environ  import *

import time

""" Loading Battery Class """
from battery import Battery

## Data Preparation
Here:
* We read the data
* Fix the date to have a **Date format** which is DD/MM/YY HH:MM
* Check if there are missing values and fill them up (incase)
* Set 'timesteps' or 'Datetime' to be the index

In [ ]:
""" Read the data """

df = pd.read_csv("load_and_dam_data.csv")
df.columns = ['Datetime', 'load_kW', 'dam_price']
df['Datetime'] = [datetime.strptime(x, '%d/%m/%y %H:%M') for x in df['Datetime']]
pd.set_option('display.max_rows', None)

# filling data using the median function
def fillmissingvalues(data):
    median = data.median()
    data.fillna(median, inplace = True)
    
    return data
df = fillmissingvalues(df)
#setting the datetime as an index
dataset = df
dataset = df.set_index("Datetime")
dataset.index = pd.to_datetime(dataset.index)
print(dataset.info())

### Series of functions incase they're needed to be used:
* A function that resamples the data
* A function that normalizes the data

In [ ]:
""" Function to resample the data """

def reSampleByValue(data, dateValue = 'H'):
    if (dateValue == 'D') or (dateValue == 'W') or (dateValue == 'M') or (dateValue == 'Y'):
    # Date Related like D: Daily, M: Monthly, Y: Yearly
        resampled_data = data.iloc[:,:].resample(dateValue).sum()
        
    elif (dateValue == 'H') or (dateValue == '15Min'):
    # Time Related like hourly and by 15 minutes
        if (dateValue == 'H'):
            # every hour
            by_hour = data.iloc[:,:].resample('H').sum()
            resampled_data = by_hour.groupby(by_hour.index).sum()
        elif (dateValue == '15Min'):
            #every 15 minutes
            resampled_data = data.iloc[:,:]
        
    else:
            resampled_data = 'Sorry, the values you can use are: D, W, M, Y, H and 15Min'
    
    # returns the data the way you chose to represent it (by hour, day, week, etc...)
    return resampled_data


dataset_day = reSampleByValue(dataset, dateValue = 'D')
dataset_hour = reSampleByValue(dataset, dateValue = 'H')


""" normalizing the data """

def theDataNormalizer(data):
    scaler = MinMaxScaler(feature_range = (0, 1))
    data_scaled = scaler.fit_transform(data)
    
    return data_scaled

#test
data_norm = theDataNormalizer(dataset.iloc[:,:])

Which day is going to be plotted and analysed? 

In [ ]:
d=21 #day number

# Step One: Day ahead bid commitment:
* First line creates a series of random error uniformly destributed between -1 and 1 (kW) and it's uncorrelated with time U(-1, 1)
* Second line is the Synthetic Day-ahead forcast where the random error (U(-1, 1)) is added to the load of the next day

In [ ]:
#random uniform error between -1 and 1
#dataset['randError'] = ss.uniform.rvs(size=2977, loc = -1, scale = 2, random_state = 1995)
#random Normal error between -1 and 1
#dataset['randError'] = ss.norm.rvs(size=2977, loc = 0, scale = 1/3, random_state = 1995)
# random gamma error between -1 and 1
dataset['randError'] = ss.gamma.rvs(a = 2, size=2977, loc = 0, scale=0.17, random_state= 1995)

# Synthetic Day-ahead Forecast 
dataset['SyntheticLoadForecast'] = dataset['load_kW'] + dataset['randError']

### Day Index column

Here an extra column is added to differentiate days between eachother.
There exist 31 days with the current dataset and each day has 96 slots so each day will have like:

Day 1: t=1 dayIndex = 1,
       t=2 dayIndex = 1,
       ...
       t=96 dayIndex = 1
...
...

Day 20: t=1 dayIndex = 20,
        t=2 dayIndex = 20,
        ...
        t=96 dayIndex = 20
...
...

In [ ]:
dataset_day = reSampleByValue(dataset, dateValue = 'D')
dataset['dayIndex'] = None
dayIndex = []
for i in range(1, len(dataset_day)+1): 
    for j in range(96):
        dayIndex.append(i)
        
for a in range(len(dataset)):
    dataset.iloc[a,4] = dayIndex[a]

In [ ]:
# making np arrays (it is unnecessary but just makes my life easier)
p_forecast = dataset.loc[dataset['dayIndex'] == d].SyntheticLoadForecast.values
dam_price = dataset.loc[dataset['dayIndex'] == d].dam_price.values

#p_commit = []
#for i in range(96):
#    if dataset.loc[dataset['dayIndex'] == d].SyntheticLoadForecast.values[i]<0:
#        p_commit.append(0)
#    else:
#        p_commit.append(dataset.loc[dataset['dayIndex'] == d].SyntheticLoadForecast.values[i])


#deviation = dev_table
        
# Duration of a market dispatch time interval
MarketTime = 1 # 1 = 1 hour, 0.25 = 15 minutes
# Imbalance cost
imb_cost = 0.02

### Graph of committed power at every time t for a single day
where the committed power is assumed to be the forecasted power.

# Step Two: The next day comes

We will see what the objective function will look like when there is no battery.

In [ ]:
p_actualLoad = dataset.loc[dataset['dayIndex'] == d].load_kW.values
p_commit = dataset.loc[dataset['dayIndex'] == d].SyntheticLoadForecast.values
dam_price = dataset.loc[dataset['dayIndex'] == d].dam_price.values


""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""" Objective Function """""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""
objfun = 0
for t in range(96):
    objfun += ((dam_price[t] * p_actualLoad[t]) + (imb_cost * abs((p_actualLoad[t] - p_commit[t]))))*MarketTime    

In [ ]:
objfun

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
ax1.plot(dataset.loc[dataset['dayIndex'] == d].index, p_actualLoad - p_commit , "-H", color='black', linewidth=3, label = 'residuals') # residuals

ax1.legend(loc = 0, prop={'size': 18})

ax1.set_xlabel('timesteps (96)', fontsize = 20)
ax1.set_ylabel('residuals', fontsize = 20)

#defining display layout 
plt.tight_layout()

# Save figure
#fig.savefig('Results/StrategyWithNoForecast/Battery-operation/Day{0}/Norm_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyWithNoForecast/Battery-operation/Day{0}/Uniform_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyWithNoForecast/Battery-operation/Day{0}/gamma_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

# show graph
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
kwargs = dict(hist_kws={'alpha':.6}, kde_kws={'linewidth':2})
sns.distplot(p_actualLoad - p_commit, color="dodgerblue", label="Residuals Distribution",  **kwargs)
plt.legend(loc = 0, prop={'size': 18});

# Save figure
#fig.savefig('Results/StrategyWithNoForecast/Battery-operation/Day{0}/Norm_dist_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyWithNoForecast/Battery-operation/Day{0}/Uniform_dist_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyWithNoForecast/Battery-operation/Day{0}/gamma_dist_RTresiduals_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')

In [ ]:
imbalance_cost = []
bidding_cost = []
for t in range(len(dataset.loc[dataset['dayIndex'] == d])):
    imb_res = imb_cost *(p_actualLoad[t] - p_commit[t])
    bid_res = p_commit[t] * dam_price[t]
    imbalance_cost.append(imb_res)
    bidding_cost.append(bid_res)

In [ ]:
fig, ax1 = plt.subplots(figsize = (16,8))
ax2 = ax1.twinx()

lns1 = ax1.plot(dataset.loc[dataset['dayIndex'] == d].index, bidding_cost, "-", color= '#984ea3',linewidth=3, label = "bidding cost")
lns2 = ax2.plot(dataset.loc[dataset['dayIndex'] == d].index, imbalance_cost, "-s", color= '#f781bf',linewidth=3, label = "imbalance cost")

# added these three lines
lns = lns1+lns2
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc = 3, prop={'size': 18})

ax1.set_xlabel('timesteps (96)', fontsize=18)
ax1.set_ylabel('bidding costs', fontsize=18)
ax2.set_ylabel('Imbalances', fontsize=18)

#defining display layout 
plt.tight_layout()

# Save figure
#fig.savefig('Results/StrategyWithNoForecast/Battery-operation/Day{0}/Uniform_bidding_vs_imbalance_cost_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyWithNoForecast/Battery-operation/Day{0}/Norm_bidding_vs_imbalance_cost_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')
#fig.savefig('Results/StrategyWithNoForecast/Battery-operation/Day{0}/gamma_bidding_vs_imbalance_cost_Day{0}.png'.format(d), dpi=300, bbox_inches='tight')


# show graph
plt.show()

bidding vs imbalance cost at every time t for a single day

In [ ]:
# forecast no battery
total_bidding_cost = 0
total_imbalance_cost = 0
total_overall_cost = objfun

for t in range(len(dataset.loc[dataset['dayIndex'] == d])):
    total_bidding_cost += bidding_cost[t]
    total_imbalance_cost += abs(imbalance_cost[t])
    
total_overall_cost,total_bidding_cost,total_imbalance_cost
